In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, models, transforms, utils
import torchvision.transforms.functional as TF

from tqdm import tqdm
import numpy as np
import json
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import skimage
import skimage.io
import skimage.transform
from PIL import Image
import time
import os
from os.path import join, exists
import copy
import random
from collections import OrderedDict
from sklearn.metrics import r2_score

from torch.nn import functional as F
from torchvision.models import Inception3

import sys
sys.path.append("/home/ubuntu/deepsolar/src/")

from inception_modified import InceptionSegmentation
from image_dataset import ImageFolderModified, ImageFolderModifiedEvaluation


In [7]:
# Configuration
# directory for loading training/validation/test data
mode = 'val' # 'eval' or 'val'
# data_dir = '/home/ubuntu/deepsolar/data/ds-usa/eval'  #'/home/ubuntu/projects/deepsolar/deepsolar_dataset_toy/test'
data_dir = '/home/ubuntu/deepsolar/data/ds-france/google/ft_5000/val'
# data_dir = '/home/ubuntu/deepsolar/data/ds-france/google/ft_100/val/' 
classification_path = '/home/ubuntu/deepsolar/checkpoint/ft_5000_classification_tune_sweep_best_models/psel_0-6956991453638643_lr_0-00016152257047812214_lr_decay_rate_0-3042236746546522_weight_decay_0-16265604808312772_epoch__8_last.tar'
segmentation_path = '/home/ubuntu/deepsolar/checkpoint/ft_5000_segmentation_level_2_tune_sweep_best_models/psel_0-6956991453638643_lr_0-0002431815776062007_lr_decay_rate_0-3461895018894605_weight_decay_0-07219288691242215_epoch__0_last.tar'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_size = 299
batch_size = 1   # must be 1 for testing segmentation
class_threshold = 0.5  # threshold probability to identify am image as positive
true_cam_threshold = 0.5
seg_threshold = 0.82    # threshold to identify a pixel as positive.
level = 2
cam_filepath = 'CAM_ft_5000_finetune_test.pickle' 


In [8]:
transform_test = transforms.Compose([
                 transforms.Resize((input_size, input_size)),
                 transforms.ToTensor(),
                 transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
                 ])


In [9]:
def precision(stats):
    return (stats['TP'] + 0.00001) * 1.0 / (stats['TP'] + stats['FP'] + 0.00001)
                           
def recall(stats):
    return (stats['TP'] + 0.00001) * 1.0 / (stats['TP'] + stats['FN'] + 0.00001)

def f1_score(stats):
    prec = precision(stats)
    rec = recall(stats)
    print('precision: ' + str(prec))
    print('recall: ' + str(rec)) 
    print('stats: ')
    print(stats)
    f1 = (prec * rec) / (prec + rec)
    return f1

def metrics(stats):
    """
    Self-defined metrics function to evaluate and compare models
    stats: {'TP': TP, 'FP': FP, 'TN': TN, 'FN': FN}
    return: must be a single number """
    accuracy = (stats['TP'] + stats['TN']) / (stats['TP'] + stats['FP'] + stats['TN'] + stats['FN'])
    return accuracy


In [5]:
import matplotlib.pyplot as plt
def test_model(model, dataloader, metrics, class_threshold, seg_threshold, true_cam_threshold):
    stats = {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0}
    estimated_area = 0
    true_area = 0
    model.eval()
    CAM_list = []
    iou = []
    count = 0
    for inputs, labels, paths in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.set_grad_enabled(False):
            _, outputs, CAM = model(inputs, testing=True)   # CAM is a 1 x 35 x 35 activation map
            prob = F.softmax(outputs, dim=1)
            preds = prob[:, 1] >= class_threshold

        CAM = CAM.squeeze(0).cpu().numpy()   # transform tensor into numpy array
        for i in range(preds.size(0)):
            predicted_label = preds[i] 
            # if labels[i]==1: # oracle to see how much improving classification would improve estimation
            if predicted_label.cpu().item():
                CAM_list.append((CAM, paths[i]))        # only use the generated CAM if it is predicted to be 1
                CAM_rescaled = (CAM - CAM.min()) / (CAM.max() - CAM.min())    # get predicted area
                CAM_pred = CAM_rescaled > seg_threshold
                pred_pixel_area = np.sum(CAM_pred)
                estimated_area += pred_pixel_area
                
            else:
                CAM_list.append((np.zeros_like(CAM), paths[i]))  # otherwise the CAM is a totally black one
                CAM_pred = np.zeros_like(CAM)

            if labels[i] == 1:
                # calculate true area
                img_path = os.path.splitext(paths[i])
                true_seg_path = img_path[0] + '_true_seg' + img_path[1]
                true_seg_img = Image.open(true_seg_path)
                transform = transforms.Compose([transforms.ToTensor()])
                true_seg = transform(true_seg_img)
                true_seg = true_seg.squeeze(0).cpu().numpy()
                true_pixel_area = np.sum(true_seg)
                true_pixel_area = true_pixel_area * (35. * 35.) / (true_seg.shape[0] * true_seg.shape[1])
                true_area += true_pixel_area
                CAM_true = skimage.transform.resize(true_seg, (35,35))
                CAM_true = CAM_true > true_cam_threshold

            else:
                CAM_true = np.zeros_like(CAM)

        intersection = CAM_true * CAM_pred
        union = CAM_true + CAM_pred
        if union.sum() > 0:
            iou.append(intersection.sum() / float(union.sum()))

        stats['TP'] += torch.sum((preds == 1) * (labels == 1)).cpu().item()
        stats['TN'] += torch.sum((preds == 0) * (labels == 0)).cpu().item()
        stats['FP'] += torch.sum((preds == 1) * (labels == 0)).cpu().item()
        stats['FN'] += torch.sum((preds == 0) * (labels == 1)).cpu().item()


    metric_value = metrics(stats)
    print('IOU: ' + str(np.mean(iou)))
    return stats, metric_value, CAM_list, estimated_area, true_area


In [8]:
def run_val(segmentation_path, seg_threshold, cam_filepath):
    mode = 'val'
    if mode=='eval':
        print('evaluating on test set')
        dataset_test = ImageFolderModifiedEvaluation(data_dir, transform_test)
    else:
        dataset_test = ImageFolderModified(data_dir, transform_test)
    dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers=4)

    # model
    model = InceptionSegmentation(num_outputs=2, level=level)
    model.load_existing_params(segmentation_path)

    model = model.to(device)

    stats, metric_value, CAM_list, estimated_area, true_area = test_model(model, dataloader_test, metrics, class_threshold=class_threshold, seg_threshold=seg_threshold, true_cam_threshold=true_cam_threshold)
    precision = (stats['TP'] + 0.00001) * 1.0 / (stats['TP'] + stats['FP'] + 0.00001)
    recall = (stats['TP'] + 0.00001) * 1.0 / (stats['TP'] + stats['FN'] + 0.00001)
    print('metric value: '+str(metric_value))
    print('precision: ' + str(round(precision, 4)))
    print('recall: ' + str(round(recall, 4)))
    print('estimated_area: ' + str(estimated_area))
    print('true_area: ' + str(true_area))
    print('ratio (est / true): ' + str(estimated_area / true_area))
    print('error ((est - true) / true): ' + str((estimated_area - true_area) / true_area)) 

    with open(cam_filepath, 'wb') as f:
        pickle.dump(CAM_list, f)


In [13]:
for threshold in [0.70, 0.73, 0.75, 0.77]:
    print('###################Threshold:{}###################'.format(threshold))
    run_val(segmentation_path, threshold, cam_filepath)

###################Threshold:0.7###################


/opt/conda/envs/pytorch/lib/python3.9/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_5000_segmentation_level_2_tune_sweep_best_models/psel_0-6956991453638643_lr_0-0002431815776062007_lr_decay_rate_0-3461895018894605_weight_decay_0-07219288691242215_epoch__0_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:45<00:00, 43.62it/s]


IOU: 0.39307012590848595
metric value: 0.9735
precision: 0.9693
recall: 0.978
estimated_area: 27479
true_area: 23383.006718749966
ratio (est / true): 1.1751696576285722
error ((est - true) / true): 0.17516965762857215
###################Threshold:0.73###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_5000_segmentation_level_2_tune_sweep_best_models/psel_0-6956991453638643_lr_0-0002431815776062007_lr_decay_rate_0-3461895018894605_weight_decay_0-07219288691242215_epoch__0_last.tar


...

IOU: 0.37725766247761694
metric value: 0.9735
precision: 0.9693
recall: 0.978
estimated_area: 22275
true_area: 23383.006718749966
ratio (est / true): 0.9526148740374993
error ((est - true) / true): -0.047385125962500656
###################Threshold:0.75###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_5000_segmentation_level_2_tune_sweep_best_models/psel_0-6956991453638643_lr_0-0002431815776062007_lr_decay_rate_0-3461895018894605_weight_decay_0-07219288691242215_epoch__0_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:46<00:00, 43.07it/s]


IOU: 0.3647871177832733
metric value: 0.9735
precision: 0.9693
recall: 0.978
estimated_area: 19399
true_area: 23383.006718749966
ratio (est / true): 0.8296195708845544
error ((est - true) / true): -0.17038042911544557
###################Threshold:0.77###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_5000_segmentation_level_2_tune_sweep_best_models/psel_0-6956991453638643_lr_0-0002431815776062007_lr_decay_rate_0-3461895018894605_weight_decay_0-07219288691242215_epoch__0_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:45<00:00, 43.70it/s]


IOU: 0.34470351582065245
metric value: 0.9735
precision: 0.9693
recall: 0.978
estimated_area: 16770
true_area: 23383.006718749966
ratio (est / true): 0.7171874943932149
error ((est - true) / true): -0.282812505606785


In [16]:
segmentation_path = '/home/ubuntu/deepsolar/checkpoint/ft_1000_segmentation_level_2_tune_sweep_best_models/psel_0-7246077310839052_lr_0-0006188732400968487_lr_decay_rate_0-33457774028832415_weight_decay_0-05248446740883417_epoch__10_last.tar'
cam_filepath = 'CAM_list.pickle'
for threshold in [0.83, 0.85, 0.87]:
    print('###################Threshold:{}###################'.format(threshold))
    run_val(segmentation_path, threshold, cam_filepath)

###################Threshold:0.83###################


/opt/conda/envs/pytorch/lib/python3.9/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_1000_segmentation_level_2_tune_sweep_best_models/psel_0-7246077310839052_lr_0-0006188732400968487_lr_decay_rate_0-33457774028832415_weight_decay_0-05248446740883417_epoch__10_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:46<00:00, 43.21it/s]


IOU: 0.16438602448647707
metric value: 0.9725
precision: 0.9637
recall: 0.982
estimated_area: 32426
true_area: 23383.006718749966
ratio (est / true): 1.386733553559594
error ((est - true) / true): 0.3867335535595939
###################Threshold:0.85###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_1000_segmentation_level_2_tune_sweep_best_models/psel_0-7246077310839052_lr_0-0006188732400968487_lr_decay_rate_0-33457774028832415_weight_decay_0-05248446740883417_epoch__10_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:46<00:00, 43.43it/s]


IOU: 0.16351571116811026
metric value: 0.9725
precision: 0.9637
recall: 0.982
estimated_area: 25685
true_area: 23383.006718749966
ratio (est / true): 1.0984472745074376
error ((est - true) / true): 0.09844727450743752
###################Threshold:0.87###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_1000_segmentation_level_2_tune_sweep_best_models/psel_0-7246077310839052_lr_0-0006188732400968487_lr_decay_rate_0-33457774028832415_weight_decay_0-05248446740883417_epoch__10_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:45<00:00, 44.32it/s]


IOU: 0.15839995024739792
metric value: 0.9725
precision: 0.9637
recall: 0.982
estimated_area: 20135
true_area: 23383.006718749966
ratio (est / true): 0.8610954203701481
error ((est - true) / true): -0.1389045796298519


In [17]:
for threshold in [0.86]:
    print('###################Threshold:{}###################'.format(threshold))
    run_val(segmentation_path, threshold, cam_filepath)

###################Threshold:0.86###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_1000_segmentation_level_2_tune_sweep_best_models/psel_0-7246077310839052_lr_0-0006188732400968487_lr_decay_rate_0-33457774028832415_weight_decay_0-05248446740883417_epoch__10_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:46<00:00, 43.30it/s]


IOU: 0.16172722972753958
metric value: 0.9725
precision: 0.9637
recall: 0.982
estimated_area: 22774
true_area: 23383.006718749966
ratio (est / true): 0.9739551578599331
error ((est - true) / true): -0.026044842140066887


In [18]:
segmentation_path = '/home/ubuntu/deepsolar/checkpoint/ft_500_segmentation_level_2_tune_sweep_best_models/psel_0-8188110871270571_lr_0-0008709800586192774_lr_decay_rate_0-4240342150652191_weight_decay_0-2204940780344005_epoch__9_last.tar'
cam_filepath = 'CAM_list.pickle'
for threshold in [0.6, 0.7, 0.8, 0.9]:
    print('###################Threshold:{}###################'.format(threshold))
    run_val(segmentation_path, threshold, cam_filepath)

###################Threshold:0.6###################


/opt/conda/envs/pytorch/lib/python3.9/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_500_segmentation_level_2_tune_sweep_best_models/psel_0-8188110871270571_lr_0-0008709800586192774_lr_decay_rate_0-4240342150652191_weight_decay_0-2204940780344005_epoch__9_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:46<00:00, 43.37it/s]


IOU: 0.3096922064267837
metric value: 0.953
precision: 0.9494
recall: 0.957
estimated_area: 88825
true_area: 23383.006718749966
ratio (est / true): 3.798698818692744
error ((est - true) / true): 2.798698818692744
###################Threshold:0.7###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_500_segmentation_level_2_tune_sweep_best_models/psel_0-8188110871270571_lr_0-0008709800586192774_lr_decay_rate_0-4240342150652191_weight_decay_0-2204940780344005_epoch__9_last.tar


...

IOU: 0.4083960328243958
metric value: 0.953
precision: 0.9494
recall: 0.957
estimated_area: 41117
true_area: 23383.006718749966
ratio (est / true): 1.7584137273086358
error ((est - true) / true): 0.7584137273086358
###################Threshold:0.8###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_500_segmentation_level_2_tune_sweep_best_models/psel_0-8188110871270571_lr_0-0008709800586192774_lr_decay_rate_0-4240342150652191_weight_decay_0-2204940780344005_epoch__9_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:46<00:00, 43.41it/s]


IOU: 0.39656591422476023
metric value: 0.953
precision: 0.9494
recall: 0.957
estimated_area: 18102
true_area: 23383.006718749966
ratio (est / true): 0.7741519393861644
error ((est - true) / true): -0.22584806061383556
###################Threshold:0.9###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_500_segmentation_level_2_tune_sweep_best_models/psel_0-8188110871270571_lr_0-0008709800586192774_lr_decay_rate_0-4240342150652191_weight_decay_0-2204940780344005_epoch__9_last.tar


 28%|███████████████████████████                                                                     | 565/2000 [00:10<00:26, 54.49it/s]


KeyboardInterrupt: 

In [19]:
for threshold in [0.73, 0.75, 0.77]:
    print('###################Threshold:{}###################'.format(threshold))
    run_val(segmentation_path, threshold, cam_filepath)

###################Threshold:0.73###################


/opt/conda/envs/pytorch/lib/python3.9/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_500_segmentation_level_2_tune_sweep_best_models/psel_0-8188110871270571_lr_0-0008709800586192774_lr_decay_rate_0-4240342150652191_weight_decay_0-2204940780344005_epoch__9_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:46<00:00, 43.12it/s]


IOU: 0.4184833724038731
metric value: 0.953
precision: 0.9494
recall: 0.957
estimated_area: 32280
true_area: 23383.006718749966
ratio (est / true): 1.3804897029822887
error ((est - true) / true): 0.3804897029822886
###################Threshold:0.75###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_500_segmentation_level_2_tune_sweep_best_models/psel_0-8188110871270571_lr_0-0008709800586192774_lr_decay_rate_0-4240342150652191_weight_decay_0-2204940780344005_epoch__9_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:46<00:00, 43.36it/s]


IOU: 0.4210226666473637
metric value: 0.953
precision: 0.9494
recall: 0.957
estimated_area: 27415
true_area: 23383.006718749966
ratio (est / true): 1.1724326272385206
error ((est - true) / true): 0.17243262723852051
###################Threshold:0.77###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_500_segmentation_level_2_tune_sweep_best_models/psel_0-8188110871270571_lr_0-0008709800586192774_lr_decay_rate_0-4240342150652191_weight_decay_0-2204940780344005_epoch__9_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:46<00:00, 42.99it/s]


IOU: 0.41591631881830365
metric value: 0.953
precision: 0.9494
recall: 0.957
estimated_area: 23344
true_area: 23383.006718749966
ratio (est / true): 0.9983318347713304
error ((est - true) / true): -0.0016681652286695962


In [20]:
segmentation_path = '/home/ubuntu/deepsolar/checkpoint/ft_100_segmentation_level_2_sweep_best_models/psel_0_lr_0-0004228521209106932_lr_decay_rate_0-1180038312075688_weight_decay_0-2207810660245898_epoch__8_last.tar'
cam_filepath = 'CAM_list.pickle'

for threshold in [0.6, 0.7, 0.8, 0.9]:
    print('###################Threshold:{}###################'.format(threshold))
    run_val(segmentation_path, threshold, cam_filepath)

###################Threshold:0.6###################


/opt/conda/envs/pytorch/lib/python3.9/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_100_segmentation_level_2_sweep_best_models/psel_0_lr_0-0004228521209106932_lr_decay_rate_0-1180038312075688_weight_decay_0-2207810660245898_epoch__8_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:45<00:00, 43.71it/s]


IOU: 0.30904415738352403
metric value: 0.868
precision: 0.8031
recall: 0.975
estimated_area: 52527
true_area: 23383.006718749966
ratio (est / true): 2.2463749265350272
error ((est - true) / true): 1.246374926535027
###################Threshold:0.7###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_100_segmentation_level_2_sweep_best_models/psel_0_lr_0-0004228521209106932_lr_decay_rate_0-1180038312075688_weight_decay_0-2207810660245898_epoch__8_last.tar


...

IOU: 0.32491767092787843
metric value: 0.868
precision: 0.8031
recall: 0.975
estimated_area: 28431
true_area: 23383.006718749966
ratio (est / true): 1.21588298468059
error ((est - true) / true): 0.21588298468059008
###################Threshold:0.8###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_100_segmentation_level_2_sweep_best_models/psel_0_lr_0-0004228521209106932_lr_decay_rate_0-1180038312075688_weight_decay_0-2207810660245898_epoch__8_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:46<00:00, 42.99it/s]


IOU: 0.2650837656273349
metric value: 0.868
precision: 0.8031
recall: 0.975
estimated_area: 14095
true_area: 23383.006718749966
ratio (est / true): 0.602788177309026
error ((est - true) / true): -0.39721182269097405
###################Threshold:0.9###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_100_segmentation_level_2_sweep_best_models/psel_0_lr_0-0004228521209106932_lr_decay_rate_0-1180038312075688_weight_decay_0-2207810660245898_epoch__8_last.tar


 80%|████████████████████████████████████████████████████████████████████████████▍                  | 1610/2000 [00:35<00:08, 45.27it/s]


KeyboardInterrupt: 

In [21]:
for threshold in [0.73, 0.75, 0.77]:
    print('###################Threshold:{}###################'.format(threshold))
    run_val(segmentation_path, threshold, cam_filepath)

###################Threshold:0.73###################


/opt/conda/envs/pytorch/lib/python3.9/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_100_segmentation_level_2_sweep_best_models/psel_0_lr_0-0004228521209106932_lr_decay_rate_0-1180038312075688_weight_decay_0-2207810660245898_epoch__8_last.tar


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:46<00:00, 43.27it/s]


IOU: 0.3153304174582409
metric value: 0.868
precision: 0.8031
recall: 0.975
estimated_area: 23378
true_area: 23383.006718749966
ratio (est / true): 0.9997858821660454
error ((est - true) / true): -0.00021411783395467005
###################Threshold:0.75###################
Loaded existing model parameters from: /home/ubuntu/deepsolar/checkpoint/ft_100_segmentation_level_2_sweep_best_models/psel_0_lr_0-0004228521209106932_lr_decay_rate_0-1180038312075688_weight_decay_0-2207810660245898_epoch__8_last.tar


 45%|██████████████████████████████████████████▊                                                     | 893/2000 [00:16<00:20, 54.33it/s]


KeyboardInterrupt: 

In [11]:
import matplotlib.pyplot as plt
def test_model(model, dataloader, metrics, class_threshold, seg_threshold, true_cam_threshold):
    stats = {'TP': 0, 'FP': 0, 'TN': 0, 'FN': 0}
    estimated_area = 0
    true_area = 0
    model.eval()
    CAM_list = []
    true_CAM_list = []
    iou = []
    for inputs, labels, paths in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.set_grad_enabled(False):
            _, outputs, CAM = model(inputs, testing=True)   # CAM is a 1 x 35 x 35 activation map
            prob = F.softmax(outputs, dim=1)
            preds = prob[:, 1] >= class_threshold

        CAM = CAM.squeeze(0).cpu().numpy()   # transform tensor into numpy array
        for i in range(preds.size(0)):
            predicted_label = preds[i] 
            # if labels[i]==1: # oracle to see how much improving classification would improve estimation
            if predicted_label.cpu().item():
                CAM_list.append((CAM, paths[i]))        # only use the generated CAM if it is predicted to be 1
                CAM_rescaled = (CAM - CAM.min()) / (CAM.max() - CAM.min())    # get predicted area
                CAM_pred = CAM_rescaled > seg_threshold
                pred_pixel_area = np.sum(CAM_pred)
                estimated_area += pred_pixel_area
                
            else:
                CAM_pred = np.zeros_like(CAM)
                CAM_list.append((np.zeros_like(CAM), paths[i]))  # otherwise the CAM is a totally black one

            if labels[i] == 1:
                # calculate true area
                img_path = os.path.splitext(paths[i])
                true_seg_path = img_path[0] + '_true_seg' + img_path[1]
                true_seg_img = Image.open(true_seg_path)
                transform = transforms.Compose([transforms.ToTensor()])
                true_seg = transform(true_seg_img)
                true_seg = true_seg.squeeze(0).cpu().numpy()
                true_pixel_area = np.sum(true_seg)
                true_pixel_area = true_pixel_area * (35. * 35.) / (true_seg.shape[0] * true_seg.shape[1])
                true_area += true_pixel_area
                CAM_true = skimage.transform.resize(true_seg, (35,35))
                CAM_true = CAM_true > true_cam_threshold

            else:
                CAM_true = np.zeros_like(CAM)
                true_CAM_list.append((CAM_true, paths[i]))

        intersection = CAM_true * CAM_pred
        union = CAM_true + CAM_pred
        if union.sum() > 0:
            iou.append(intersection.sum() / float(union.sum()))

        stats['TP'] += torch.sum((preds == 1) * (labels == 1)).cpu().item()
        stats['TN'] += torch.sum((preds == 0) * (labels == 0)).cpu().item()
        stats['FP'] += torch.sum((preds == 1) * (labels == 0)).cpu().item()
        stats['FN'] += torch.sum((preds == 0) * (labels == 1)).cpu().item()

    mean_iou = np.mean(iou)
    metric_value = metrics(stats)
    return stats, mean_iou, CAM_list, true_CAM_list, estimated_area, true_area


In [13]:
def run_eval(segmentation_path, seg_threshold, cam_filepath, data_dir, mode):
    if mode=='eval':
        print('evaluating on test set')
        dataset_test = ImageFolderModifiedEvaluation(data_dir, transform_test)
    else:
        dataset_test = ImageFolderModified(data_dir, transform_test)
    dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False, num_workers=4)

    # model
    model = InceptionSegmentation(num_outputs=2, level=level)
    model.load_existing_params(segmentation_path)

    model = model.to(device)

    stats, iou, CAM_list, true_CAM_list, estimated_area, true_area = test_model(model, dataloader_test, metrics, class_threshold=class_threshold, seg_threshold=seg_threshold, true_cam_threshold=true_cam_threshold)

    cams = {'true': true_CAM_list,
            'pred': CAM_list}
    # dump CAM to file
    with open(cam_filepath, 'wb') as f:
        pickle.dump(cams, f)
    print("Saved CAMs to " + cam_filepath)

    prec = precision(stats)
    rec = recall(stats)
    acc = metrics(stats)
    area_fractional_error = (estimated_area - true_area) / true_area
    
    print('accuracy: '+str(acc))
    print('precision: ' + str(round(prec, 4)))
    print('recall: ' + str(round(rec, 4)))
    print('iou: ' + str(iou))
    print('estimated_area: ' + str(estimated_area))
    print('true_area: ' + str(true_area))
    print('error ((est - true) / true): ' + str(area_fractional_error))

    return (acc, prec, rec, iou, area_fractional_error)


In [15]:
NAMES = ['baseline', 'ft_100', 'ft_500', 'ft_1000', 'ft_5000']
SEGMENTATION_PATHS = [
    '/home/ubuntu/deepsolar/models/deepsolar_seg_pretrained.pth',
    '/home/ubuntu/deepsolar/checkpoint/ft_100_segmentation_level_2_sweep_best_models/psel_0_lr_0-0004228521209106932_lr_decay_rate_0-1180038312075688_weight_decay_0-2207810660245898_epoch__8_last.tar',
    '/home/ubuntu/deepsolar/checkpoint/ft_500_segmentation_level_2_tune_sweep_best_models/psel_0-8188110871270571_lr_0-0008709800586192774_lr_decay_rate_0-4240342150652191_weight_decay_0-2204940780344005_epoch__9_last.tar',
    '/home/ubuntu/deepsolar/checkpoint/ft_1000_segmentation_level_2_tune_sweep_best_models/psel_0-7246077310839052_lr_0-0006188732400968487_lr_decay_rate_0-33457774028832415_weight_decay_0-05248446740883417_epoch__10_last.tar',
    '/home/ubuntu/deepsolar/checkpoint/ft_5000_segmentation_level_2_tune_sweep_best_models/psel_0-6956991453638643_lr_0-0002431815776062007_lr_decay_rate_0-3461895018894605_weight_decay_0-07219288691242215_epoch__0_last.tar'
]
SEGMENTATION_THRESHOLDS = [0.37, 0.73, 0.77, 0.86, 0.73]
FR_EVAL_DATA = '/home/ubuntu/deepsolar/data/ds-france/google/ft_eval'
US_EVAL_DATA = '/home/ubuntu/deepsolar/data/ds-usa/eval'

NAMES = ['baseline']
SEGMENTATION_PATHS = [
    '/home/ubuntu/deepsolar/models/deepsolar_seg_pretrained.pth',
]
SEGMENTATION_THRESHOLDS = [0.37]
FR_EVAL_DATA = '/home/ubuntu/deepsolar/data/ds-france/google/ft_eval'
US_EVAL_DATA = '/home/ubuntu/deepsolar/data/ds-usa/eval'



In [16]:
names = []
accs = []
precs = []
recs = []
ious = []
area_fractional_errors = []
ds_names = []

for name, path, seg_threshold in zip(NAMES, SEGMENTATION_PATHS, SEGMENTATION_THRESHOLDS):
    for ds_name, data_dir, mode in zip(['fr', 'us'], [FR_EVAL_DATA, US_EVAL_DATA], ['val', 'eval']):
        print("############################ Evaluating {} on {} eval dataset #######################################".format(name, ds_name))
        cam_filepath = 'CAM_list_' + name + '_' + ds_name + '_eval.pickle'
        acc, prec, rec, iou, area_fractional_error = run_eval(path, seg_threshold, cam_filepath, data_dir, mode)
        names.append(name)
        accs.append(acc)
        precs.append(prec)
        recs.append(rec)
        ious.append(iou)
        area_fractional_errors.append(area_fractional_error)
        ds_names.append(ds_name)
        

############################ Evaluating baseline on fr eval dataset #######################################
Loaded existing model parameters from: /home/ubuntu/deepsolar/models/deepsolar_seg_pretrained.pth


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [03:58<00:00, 41.87it/s]


Saved CAMs to CAM_list_baseline_fr_eval.pickle
accuracy: 0.5654
precision: 0.9619
recall: 0.1362
iou: 0.06102155351183272
estimated_area: 30887
true_area: 116490.46390625011
error ((est - true) / true): -0.734853832972479
############################ Evaluating baseline on us eval dataset #######################################
evaluating on test set
Loaded existing model parameters from: /home/ubuntu/deepsolar/models/deepsolar_seg_pretrained.pth


 15%|██████████████                                                                               | 14114/93500 [04:24<24:46, 53.41it/s]

 22%|████████████████████▎                                                                        | 20435/93500 [06:21<22:44, 53.53it/s]


KeyboardInterrupt: 

In [ ]:
import pandas as pd

results = pd.Dataframe(zip(names, ds_names, accs, precs, recs, ious, area_fractional_errors), 
                    columns = ['model', 'eval ds', 'accuracy', 'precision', 'recall', 'iou', 'area_fractional_error'])

results.to_csv('results.csv')